In [29]:
import numpy as np
import rasterio
from skimage.morphology import black_tophat, square, white_tophat
from skimage.transform import rotate
import os
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob

def grayscale_raster_creation(input_MSfile, output_filename):

    with rasterio.open(input_MSfile) as f:
        metadata = f.profile
        img = np.transpose(f.read(tuple(np.arange(metadata['count']) + 1)), [1, 2, 0])[:, :, 0: 3]

    gray = np.max(img, axis=2).astype(metadata['dtype'])

    metadata['count'] = 1

    with rasterio.open(output_filename, 'w', **metadata) as dst:
        dst.write(gray[np.newaxis, :, :])

    return gray


def MBI_MSI_calculation_and_feature_map_creation(input_grayfile, output_path,
                                                 output_MBIname, output_MSIname, 
                                                 s_min, s_max, delta_s,
                                                 calc_MSI=True, write_MBI=True, write_MSI=True) -> object:

    if s_min < 3:
        raise ValueError('s_min must be greater than or equal to 3.')

    with rasterio.open(input_grayfile) as f:
        metadata = f.profile
        gray = f.read(1)

    MP_MBI_list = []
    MP_MSI_list = []
    DMP_MBI_list = []
    DMP_MSI_list = []

    for i in range(s_min, s_max + 1, 2 * delta_s):
        SE_intermediate = square(i)
        SE_intermediate[: int((i - 1) / 2), :] = 0
        SE_intermediate[int(((i - 1) / 2) + 1):, :] = 0

        SE_1 = SE_intermediate
        SE_2 = rotate(SE_1, 45, order=0, preserve_range=True).astype('uint8')
        SE_3 = rotate(SE_1, 90, order=0, preserve_range=True).astype('uint8')
        SE_4 = rotate(SE_1, 135, order=0, preserve_range=True).astype('uint8')

        MP_MBI_1 = white_tophat(gray, selem=SE_1)
        MP_MBI_2 = white_tophat(gray, selem=SE_2)
        MP_MBI_3 = white_tophat(gray, selem=SE_3)
        MP_MBI_4 = white_tophat(gray, selem=SE_4)

        if calc_MSI:
            MP_MSI_1 = black_tophat(gray, selem=SE_1)
            MP_MSI_2 = black_tophat(gray, selem=SE_2)
            MP_MSI_3 = black_tophat(gray, selem=SE_3)
            MP_MSI_4 = black_tophat(gray, selem=SE_4)

            MP_MSI_list.append(MP_MSI_1)
            MP_MSI_list.append(MP_MSI_2)
            MP_MSI_list.append(MP_MSI_3)
            MP_MSI_list.append(MP_MSI_4)

        MP_MBI_list.append(MP_MBI_1)
        MP_MBI_list.append(MP_MBI_2)
        MP_MBI_list.append(MP_MBI_3)
        MP_MBI_list.append(MP_MBI_4)

    for j in range(4, len(MP_MBI_list), 1):
        DMP_MBI_1 = np.absolute(MP_MBI_list[j] - MP_MBI_list[j - 4])
        DMP_MBI_list.append(DMP_MBI_1)

        if calc_MSI:
            DMP_MSI_1 = np.absolute(MP_MSI_list[j] - MP_MSI_list[j - 4])
            DMP_MSI_list.append(DMP_MSI_1)

    MBI = (np.sum(DMP_MBI_list, axis=0) / (4 * (((s_max - s_min) / delta_s) + 1))).astype(np.float32)

    if calc_MSI:
        MSI = (np.sum(DMP_MSI_list, axis=0) / (4 * (((s_max - s_min) / delta_s) + 1))).astype(np.float32)

    metadata['dtype'] = 'float32'
    fold_name = str(s_min) + "_" + str(s_max) + "_" + str(delta_s)
    
    # MBI SAVE
    #print(output_path, "________" , output_MBIname)
    plt.imsave(os.path.join(output_path, "mbi", output_MBIname), MBI, cmap='Greys')

    # MSI SAVE
    #print(output_path, "________" , output_MSIname)
    plt.imsave(os.path.join(output_path,"msi", output_MSIname), MSI, cmap='Greys')

    if calc_MSI:
        return MBI, MSI
    else:
        return MBI

In [32]:
if __name__ == "__main__":

    yourpath = os.getcwd()

    INPUT_PATH = r"C:\Users\mk156\Desktop\old\classfication\sat_image\ndvi\tif\19"
    OUTPUT_PATH = r"C:\Users\mk156\Desktop\old\classfication\sat_image\ndvi\tif\mbi_19"
    
    for image_name in os.listdir(INPUT_PATH):
        image_path = os.path.join(INPUT_PATH, image_name)
        gray_out_path = os.path.join(OUTPUT_PATH, image_name)
        input_gray = grayscale_raster_creation(image_path, gray_out_path)

        index_list = [[3, 50, 5], [5, 50, 7], [5, 100, 10]]
        print(image_name)

        for index in index_list:
            name = str(index[0]) + "_" + str(index[1]) + "_" + str(index[2])
            
            MBI_MSI_calculation_and_feature_map_creation(gray_out_path, OUTPUT_PATH,
                                                         image_name.split(".")[0] + "_MBI_" + name + ".png",
                                                         image_name.split(".")[0] + "_MSI_" + name + ".png",
                                                         int(index[0]), int(index[1]), int(index[2]))
            print("Done!   " + name)

        print("______________________________________________________")

19 (1).tif


C:\Users\mk156\AppData\Local\Temp\ipykernel_6272\1976765480.py:53: FutureWarning: `selem` is a deprecated argument name for `white_tophat`. It will be removed in version 1.0. Please use `footprint` instead.
  MP_MBI_1 = white_tophat(gray, selem=SE_1)
C:\Users\mk156\AppData\Local\Temp\ipykernel_6272\1976765480.py:54: FutureWarning: `selem` is a deprecated argument name for `white_tophat`. It will be removed in version 1.0. Please use `footprint` instead.
  MP_MBI_2 = white_tophat(gray, selem=SE_2)
C:\Users\mk156\AppData\Local\Temp\ipykernel_6272\1976765480.py:55: FutureWarning: `selem` is a deprecated argument name for `white_tophat`. It will be removed in version 1.0. Please use `footprint` instead.
  MP_MBI_3 = white_tophat(gray, selem=SE_3)
C:\Users\mk156\AppData\Local\Temp\ipykernel_6272\1976765480.py:56: FutureWarning: `selem` is a deprecated argument name for `white_tophat`. It will be removed in version 1.0. Please use `footprint` instead.
  MP_MBI_4 = white_tophat(gray, selem=SE

Done!   3_50_5
Done!   5_50_7
Done!   5_100_10
______________________________________________________
19 (10).tif
Done!   3_50_5
Done!   5_50_7
Done!   5_100_10
______________________________________________________
19 (11).tif
Done!   3_50_5
Done!   5_50_7
Done!   5_100_10
______________________________________________________
19 (12).tif
Done!   3_50_5
Done!   5_50_7
Done!   5_100_10
______________________________________________________
19 (13).tif
Done!   3_50_5
Done!   5_50_7
Done!   5_100_10
______________________________________________________
19 (14).tif
Done!   3_50_5
Done!   5_50_7
Done!   5_100_10
______________________________________________________
19 (15).tif
Done!   3_50_5
Done!   5_50_7
Done!   5_100_10
______________________________________________________
19 (16).tif
Done!   3_50_5
Done!   5_50_7
Done!   5_100_10
______________________________________________________
19 (17).tif
Done!   3_50_5
Done!   5_50_7
Done!   5_100_10
_________________________________________

In [36]:
MBI_PATH = r"C:\Users\mk156\Desktop\old\classfication\sat_image\ndvi\tif\mbi_19"
OUT_PATH = r"C:\Users\mk156\Desktop\old\classfication\sat_image\ndvi\tif\mbi_19\meow"
os.chdir(MBI_PATH)

# image_list generation
image_list = glob("mbi/*.png")
print("image detected: ", len(image_list))

for image in image_list:
    MyImg = Image.open(image) 
    pixels = MyImg.load() 
    modified_pixel = 0
    
    for i in range(MyImg.size[0]):    
        for j in range(MyImg.size[1]):
              
            # 0 black ~ 255 white
            if pixels[i,j][0] < 220:
                pixels[i,j] = (pixels[i,j][0] - 15,
                               pixels[i,j][1] - 15,
                               pixels[i,j][2] - 15,
                               255)
                
            if pixels[i,j][0] < 130:
                pixels[i,j] = (pixels[i,j][0] + 30,
                               pixels[i,j][1] + 30,  
                               pixels[i,j][2] + 30,
                               255)
                modified_pixel += 1

    #print(modified_pixel)       
    print(image + "____________ Done")
    MyImg.save(os.path.join(OUT_PATH, image.split("\\")[-1]))

image detected:  84
mbi\19 (1)_MBI_3_50_5.png____________ Done
mbi\19 (1)_MBI_5_100_10.png____________ Done
mbi\19 (1)_MBI_5_50_7.png____________ Done
mbi\19 (10)_MBI_3_50_5.png____________ Done
mbi\19 (10)_MBI_5_100_10.png____________ Done
mbi\19 (10)_MBI_5_50_7.png____________ Done
mbi\19 (11)_MBI_3_50_5.png____________ Done
mbi\19 (11)_MBI_5_100_10.png____________ Done
mbi\19 (11)_MBI_5_50_7.png____________ Done
mbi\19 (12)_MBI_3_50_5.png____________ Done
mbi\19 (12)_MBI_5_100_10.png____________ Done
mbi\19 (12)_MBI_5_50_7.png____________ Done
mbi\19 (13)_MBI_3_50_5.png____________ Done
mbi\19 (13)_MBI_5_100_10.png____________ Done
mbi\19 (13)_MBI_5_50_7.png____________ Done
mbi\19 (14)_MBI_3_50_5.png____________ Done
mbi\19 (14)_MBI_5_100_10.png____________ Done
mbi\19 (14)_MBI_5_50_7.png____________ Done
mbi\19 (15)_MBI_3_50_5.png____________ Done
mbi\19 (15)_MBI_5_100_10.png____________ Done
mbi\19 (15)_MBI_5_50_7.png____________ Done
mbi\19 (16)_MBI_3_50_5.png____________ Done
m